## EVCS - Interactive Map Development - Melbourne and Geelong Region

In this analysis, we will built an Interactive Map that provides indications of exisiting EVCS locations, as well as the possible future charging sites. Techniques used in this analysis is based on the EVCS location datasets of Melbourne and Geelong, and Melbourne off street parking lot location dataset.

In [2]:
import pandas as pd
import folium
from geopy.distance import distance
import warnings
warnings.filterwarnings("ignore")

### Dataset Preparation

In [2]:
stations = pd.read_csv("Greater_Melbourne_and_Geelong.csv")
stations.head().T

,0,1,2,3,4
name,Geelong Supercharger,Penguin Parade Visitor's Centre,Mornington Supercharger,Kingston Village Square (DC Fast Charger),BIG4 Phillip Island Caravan Park
address,"470-510 Princes Hwy, Geelong, VIC, Australia, ...","995 Ventnor Road Summerlands VIC 3922, Australia","75 Mornington-Tyabb Rd, Mornington VIC 3931, A...","Kingston Village Square, Grubb Rd, Ocean Grove...","24 Old Bridge Drive, Newhaven VIC 3925, Australia"
longitude,144.382,145.148,145.051,144.54,145.356
latitude,-38.0652,-38.5055,-38.2343,-38.2467,-38.5168
description,NaN,NaN,NaN,NaN,NaN
parking,NaN,NaN,NaN,NaN,NaN
pricing,NaN,NaN,NaN,NaN,NaN
contact,61280152834,NaN,61280152834,1300518038,0359567227
networks,['Supercharger'],['Non-networked'],['Supercharger'],['Chargefox'],['Non-networked']
total_plugs,6,2,6,4,2


In [3]:
# Latitude and longitude values passed in from dataset
my_map = folium.Map(location = [-37.884254, 144.736465], width = 1000, height = 600)

for _, station in stations.iterrows():
    folium.Marker(location = [station["latitude"], station["longitude"]],
                 popup = station["name"],
                 tooltip = station["name"]).add_to(my_map)

my_map

### Clustered Map

In [6]:
# Clustering existing EVCSs
from folium import Marker
from folium.plugins import MarkerCluster

my_map = folium.Map(location = [-37.884254, 144.736465], width = 1000, height = 600)
mc = MarkerCluster()

for _, station in stations.iterrows():
    mc.add_child(Marker(location = [station["latitude"], station["longitude"]],
                 popup = station["name"], tooltip = station["name"]))

my_map.add_child(mc)

my_map

### Map Distinguishing lonely EVCSs

In [7]:
# Create coordinate list for storing distance between stations
coordinates = []
for _, station in stations.iterrows():
    location = [[station["latitude"], station["longitude"]]]
    coordinates += location

In [8]:
# Interate through each nearby station by the distance 
stations["whether_evcs"] = ""

for i in range(len(coordinates)):
    location1 = coordinates[i]
    evcs_counts = 0
    for j in range(len(coordinates)):       
        if i == j:
            pass
        elif distance(coordinates[j], location1) < 5: # Maximum 5km from given EVCS
            evcs_counts += 1
        else:
            evcs_counts += 0
            
    if evcs_counts > 0:
        stations["whether_evcs"].iloc[i] = "no" # No - no extra stations needed
    else:
        stations["whether_evcs"].iloc[i] = "yes" # Yes - additional stations needed

stations.head()

,name,address,longitude,latitude,description,parking,pricing,contact,networks,total_plugs,...,Plugs_Tesla,Plugs_Three_Phase,Plugs_Commando,Plugs_Type2,Plugs_Wall_AU_NZ,Plugs_Other,power_outputs_kw,source,source_date,whether_evcs
0,Geelong Supercharger,"470-510 Princes Hwy, Geelong, VIC, Australia, ...",144.382451,-38.065192,NaN,NaN,NaN,61280152834,['Supercharger'],6,...,6,0,NaN,0,0,NaN,[250.0],PlugShare_Scrape,20/08/2022,yes
1,Penguin Parade Visitor's Centre,"995 Ventnor Road Summerlands VIC 3922, Australia",145.148488,-38.505496,NaN,NaN,NaN,NaN,['Non-networked'],2,...,0,0,NaN,0,0,NaN,[25.0],PlugShare_Scrape,20/08/2022,yes
2,Mornington Supercharger,"75 Mornington-Tyabb Rd, Mornington VIC 3931, A...",145.051085,-38.234305,NaN,NaN,NaN,61280152834,['Supercharger'],6,...,6,0,NaN,0,0,NaN,[250.0],PlugShare_Scrape,20/08/2022,no
3,Kingston Village Square (DC Fast Charger),"Kingston Village Square, Grubb Rd, Ocean Grove...",144.540122,-38.246677,NaN,NaN,NaN,1300518038,['Chargefox'],4,...,0,0,NaN,2,0,NaN,"[50.0, 'unknown']",PlugShare_Scrape,20/08/2022,yes
4,BIG4 Phillip Island Caravan Park,"24 Old Bridge Drive, Newhaven VIC 3925, Australia",145.356460,-38.516829,NaN,NaN,NaN,0359567227,['Non-networked'],2,...,0,0,NaN,0,1,NaN,['unknown'],PlugShare_Scrape,20/08/2022,no


In [9]:
# Define a function to distinguish lonely stations
def select_marker_color(row):
    if row["whether_evcs"] == "yes":
        return "red"
    elif row["whether_evcs"] == "no" and row["total_plugs"] == "['unknown']":
        return "grey"
    return "green"

In [10]:
# Apply the rules to the dataset
stations["colour"] = stations.apply(select_marker_color, axis = 1)
stations.head()

,name,address,longitude,latitude,description,parking,pricing,contact,networks,total_plugs,...,Plugs_Three_Phase,Plugs_Commando,Plugs_Type2,Plugs_Wall_AU_NZ,Plugs_Other,power_outputs_kw,source,source_date,whether_evcs,colour
0,Geelong Supercharger,"470-510 Princes Hwy, Geelong, VIC, Australia, ...",144.382451,-38.065192,NaN,NaN,NaN,61280152834,['Supercharger'],6,...,0,NaN,0,0,NaN,[250.0],PlugShare_Scrape,20/08/2022,yes,red
1,Penguin Parade Visitor's Centre,"995 Ventnor Road Summerlands VIC 3922, Australia",145.148488,-38.505496,NaN,NaN,NaN,NaN,['Non-networked'],2,...,0,NaN,0,0,NaN,[25.0],PlugShare_Scrape,20/08/2022,yes,red
2,Mornington Supercharger,"75 Mornington-Tyabb Rd, Mornington VIC 3931, A...",145.051085,-38.234305,NaN,NaN,NaN,61280152834,['Supercharger'],6,...,0,NaN,0,0,NaN,[250.0],PlugShare_Scrape,20/08/2022,no,green
3,Kingston Village Square (DC Fast Charger),"Kingston Village Square, Grubb Rd, Ocean Grove...",144.540122,-38.246677,NaN,NaN,NaN,1300518038,['Chargefox'],4,...,0,NaN,2,0,NaN,"[50.0, 'unknown']",PlugShare_Scrape,20/08/2022,yes,red
4,BIG4 Phillip Island Caravan Park,"24 Old Bridge Drive, Newhaven VIC 3925, Australia",145.356460,-38.516829,NaN,NaN,NaN,0359567227,['Non-networked'],2,...,0,NaN,0,1,NaN,['unknown'],PlugShare_Scrape,20/08/2022,no,green


In [13]:
# Create an interactive map with distinguished colours and labels of station information
my_map = folium.Map(location = [-37.884254, 144.736465], width = 1000, height = 600)

for _, station in stations.iterrows():
    folium.Marker(location = [station["latitude"], station["longitude"]],
                  # Click each EVCS for popups: extra info can be added by demand
                 popup = [station["name"], station["total_plugs"]], 
                 tooltip = station["name"],
                 icon = folium.Icon(color = station["colour"])).add_to(my_map)

my_map

### New EVCS Location Recommendation

In [4]:
# Load off-street parking lot dataset
parking = pd.read_csv("Off-street_car_parking_2020.csv")
parking.head()

,Census year,Block ID,Property ID,Base property ID,Building address,CLUE small area,Parking type,Parking spaces,x coordinate,y coordinate
0,2020,1,108843,108843,2-92 Rebecca Walk MELBOURNE VIC 3000,Melbourne (CBD),Commercial,50,144.95687,-37.82121
1,2020,1,611393,611393,507-541 Flinders Street MELBOURNE VIC 3000,Melbourne (CBD),Private,29,144.95721,-37.82087
2,2020,1,611394,611394,545-557 Flinders Street MELBOURNE VIC 3000,Melbourne (CBD),Commercial,146,144.95651,-37.82098
3,2020,1,611394,611394,545-557 Flinders Street MELBOURNE VIC 3000,Melbourne (CBD),Residential,137,144.95651,-37.82098
4,2020,6,578328,573333,Carpark Federation Square 2 Swanston Street ME...,Melbourne (CBD),Commercial,450,144.96982,-37.81779


In [105]:
parking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7184 entries, 0 to 7183
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Census year       7184 non-null   int64  
 1   Block ID          7184 non-null   int64  
 2   Property ID       7184 non-null   int64  
 3   Base property ID  7184 non-null   int64  
 4   Building address  7184 non-null   object 
 5   CLUE small area   7184 non-null   object 
 6   Parking type      7184 non-null   object 
 7   Parking spaces    7184 non-null   int64  
 8   x coordinate      7183 non-null   float64
 9   y coordinate      7183 non-null   float64
dtypes: float64(2), int64(5), object(3)
memory usage: 561.4+ KB


In [73]:
# Filter based on various types of parking type
# parking = parking.loc[(parking["Parking type"] == "Commercial") | (parking["Parking type"] == "Residential")]
# parking.head()

,Census year,Block ID,Property ID,Base property ID,Building address,CLUE small area,Parking type,Parking spaces,x coordinate,y coordinate
0,2020,1,108843,108843,2-92 Rebecca Walk MELBOURNE VIC 3000,Melbourne (CBD),Commercial,50,144.95687,-37.82121
2,2020,1,611394,611394,545-557 Flinders Street MELBOURNE VIC 3000,Melbourne (CBD),Commercial,146,144.95651,-37.82098
3,2020,1,611394,611394,545-557 Flinders Street MELBOURNE VIC 3000,Melbourne (CBD),Residential,137,144.95651,-37.82098
4,2020,6,578328,573333,Carpark Federation Square 2 Swanston Street ME...,Melbourne (CBD),Commercial,450,144.96982,-37.81779
5,2020,11,103957,103957,517-537 Flinders Lane MELBOURNE VIC 3000,Melbourne (CBD),Commercial,117,144.95666,-37.81987


In [106]:
# Filter based on the parking spaces - we assume parking lots with more than 100 spaces would have potentials in new EVCSs
parking = parking.loc[parking["Parking spaces"] > 100]
parking.shape

(442, 10)

In [107]:
# Create new list for storing parking lot coordinates
parking_coordinates = []

for _, park in parking.iterrows():
    p = [[park["y coordinate"], park["x coordinate"]]]
    parking_coordinates += p

In [108]:
parking["future_ev_site"] = ""

for i in range(len(parking_coordinates)):
    location2 = parking_coordinates[i]
    evcs_counts = 0
    
    for j in range(len(coordinates)):
        
        if i == j:
            pass
        elif distance(coordinates[j], location2) < 2:
            evcs_counts += 1
        else:
            evcs_counts += 0
            
    if evcs_counts > 0:
        parking["future_ev_site"].iloc[i] = "n"
    else:
        parking["future_ev_site"].iloc[i] = "y"

parking.head()

,Census year,Block ID,Property ID,Base property ID,Building address,CLUE small area,Parking type,Parking spaces,x coordinate,y coordinate,future_ev_site
2,2020,1,611394,611394,545-557 Flinders Street MELBOURNE VIC 3000,Melbourne (CBD),Commercial,146,144.95651,-37.82098,n
3,2020,1,611394,611394,545-557 Flinders Street MELBOURNE VIC 3000,Melbourne (CBD),Residential,137,144.95651,-37.82098,n
4,2020,6,578328,573333,Carpark Federation Square 2 Swanston Street ME...,Melbourne (CBD),Commercial,450,144.96982,-37.81779,n
5,2020,11,103957,103957,517-537 Flinders Lane MELBOURNE VIC 3000,Melbourne (CBD),Commercial,117,144.95666,-37.81987,n
6,2020,11,103987,103987,550-554 Flinders Street MELBOURNE VIC 3000,Melbourne (CBD),Residential,152,144.95597,-37.82039,n


In [109]:
parking["future_ev_site"].unique()

array(['n', 'y'], dtype=object)

In [110]:
# Mapping the parking lot dataset
parking_map = folium.Map(location = [-37.82098, 144.95651], width = 1000, height = 600)

for _, park in parking.iterrows():
    folium.Marker(location = [park["y coordinate"], park["x coordinate"]],
                 popup = park["Property ID"]).add_to(parking_map)

parking_map

In [111]:
# Create a new dataset with sorted information
newdata = stations[["name", "address", "longitude", "latitude", "whether_evcs", "colour"]]
newdata.head()

,name,address,longitude,latitude,whether_evcs,colour
0,Geelong Supercharger,"470-510 Princes Hwy, Geelong, VIC, Australia, ...",144.382451,-38.065192,yes,red
1,Penguin Parade Visitor's Centre,"995 Ventnor Road Summerlands VIC 3922, Australia",145.148488,-38.505496,yes,red
2,Mornington Supercharger,"75 Mornington-Tyabb Rd, Mornington VIC 3931, A...",145.051085,-38.234305,no,green
3,Kingston Village Square (DC Fast Charger),"Kingston Village Square, Grubb Rd, Ocean Grove...",144.540122,-38.246677,yes,red
4,BIG4 Phillip Island Caravan Park,"24 Old Bridge Drive, Newhaven VIC 3925, Australia",145.356460,-38.516829,no,green


In [113]:
# New recommended EVCSs are marked as blue
new_evcs = parking.loc[parking["future_ev_site"] == "y"]
new_evcs = new_evcs[["Block ID", "Building address", "x coordinate", "y coordinate", "future_ev_site"]]
new_evcs["colour"] = "blue"
new_evcs.head()

,Block ID,Building address,x coordinate,y coordinate,future_ev_site,colour
4367,571,Melbourne Showgrounds 276-318 Epsom Road FLEMI...,144.91158,-37.78218,y,blue
6706,928,57-63 Galada Avenue PARKVILLE VIC 3052,144.93949,-37.78010,y,blue
6707,928,51-55 Galada Avenue PARKVILLE VIC 3052,144.93959,-37.78036,y,blue


In [114]:
# Rename the new dataset
new_evcs.rename(columns = {"Block ID": "name", "Building address": "address", "x coordinate": "longitude", 
                   "y coordinate": "latitude", "future_ev_site": "whether_evcs", "colour": "colour"},
        inplace = True)
new_evcs

,name,address,longitude,latitude,whether_evcs,colour
4367,571,Melbourne Showgrounds 276-318 Epsom Road FLEMI...,144.91158,-37.78218,y,blue
6706,928,57-63 Galada Avenue PARKVILLE VIC 3052,144.93949,-37.78010,y,blue
6707,928,51-55 Galada Avenue PARKVILLE VIC 3052,144.93959,-37.78036,y,blue


In [115]:
# Integarated dataset
frames = [newdata, new_evcs]
result = pd.concat(frames)
result

,name,address,longitude,latitude,whether_evcs,colour
0,Geelong Supercharger,"470-510 Princes Hwy, Geelong, VIC, Australia, ...",144.382451,-38.065192,yes,red
1,Penguin Parade Visitor's Centre,"995 Ventnor Road Summerlands VIC 3922, Australia",145.148488,-38.505496,yes,red
2,Mornington Supercharger,"75 Mornington-Tyabb Rd, Mornington VIC 3931, A...",145.051085,-38.234305,no,green
3,Kingston Village Square (DC Fast Charger),"Kingston Village Square, Grubb Rd, Ocean Grove...",144.540122,-38.246677,yes,red
4,BIG4 Phillip Island Caravan Park,"24 Old Bridge Drive, Newhaven VIC 3925, Australia",145.356460,-38.516829,no,green
...,...,...,...,...,...,...
248,Lorbek Luxury Cars,"327 Plummer St, Port Melbourne VIC 3207, Austr...",144.919764,-37.833209,no,green
249,Lorbek Luxury Cars,"30 Prohasky St, Port Melbourne VIC 3207, Austr...",144.919335,-37.832093,no,green
4367,571,Melbourne Showgrounds 276-318 Epsom Road FLEMI...,144.911580,-37.782180,y,blue
6706,928,57-63 Galada Avenue PARKVILLE VIC 3052,144.939490,-37.780100,y,blue


In [119]:
# Re-define the colour function
def select_marker_icon(row):
    if row["colour"] == "red":
        return "star"
    elif row["colour"] == "blue":
        return "heart"
    return "flash"

In [120]:
# Icon function added for map generation
result["icon"] = result.apply(select_marker_icon, axis = 1)
result.head()

,name,address,longitude,latitude,whether_evcs,colour,icon
0,Geelong Supercharger,"470-510 Princes Hwy, Geelong, VIC, Australia, ...",144.382451,-38.065192,yes,red,star
1,Penguin Parade Visitor's Centre,"995 Ventnor Road Summerlands VIC 3922, Australia",145.148488,-38.505496,yes,red,star
2,Mornington Supercharger,"75 Mornington-Tyabb Rd, Mornington VIC 3931, A...",145.051085,-38.234305,no,green,flash
3,Kingston Village Square (DC Fast Charger),"Kingston Village Square, Grubb Rd, Ocean Grove...",144.540122,-38.246677,yes,red,star
4,BIG4 Phillip Island Caravan Park,"24 Old Bridge Drive, Newhaven VIC 3925, Australia",145.356460,-38.516829,no,green,flash


In [121]:
# New map with future EVCS recommendations
new_map = folium.Map(location = [-37.884254, 144.736465], width = 1000, height = 600)

for _, r in result.iterrows():
    folium.Marker(location = [r["latitude"], r["longitude"]],
                 popup = [r["name"], r["address"]],
                 tooltip = r["name"],
                 icon = folium.Icon(icon = r["icon"], color = r["colour"])).add_to(new_map)

new_map

The END